In [ ]:
import requests
import csv
import time
from datetime import datetime

import twitter
api = twitter.Api() #pass your api key 

api_url = 'https://earthquake.usgs.gov/fdsnws/event/1/'
api_method = 'query?'
api_format = 'format=geojson'
api_starttime = '2018-08-01'
api_endtime = '2018-08-21' 
# api_minlatitude = '15'    #test data, America region
# api_minlongitude = '-120'
# api_maxlatitude = '50'
# api_maxlongitude = '-70'
api_minlatitude = '21.821'  #taiwan
api_minlongitude = '119.971'
api_maxlatitude = '25.384'
api_maxlongitude = '122.014'
url ='{0}{1}{2}&starttime={3}&minlatitude={4}&maxlatitude={5}&minlongitude={6}&maxlongitude={7}'.format(api_url,api_method,api_format,api_starttime,api_minlatitude,api_maxlatitude,api_minlongitude,api_maxlongitude)

In [ ]:
def timestamp_to_utc_strtime(timestamp):
        #将 13 位整数的毫秒时间戳转化成 utc 时间 (字符串格式，含毫秒)param timestamp: 13 位整数的毫秒时间戳 (1456402864242):return: 返回字符串格式 {str}'2016-02-25 12:21:04.242000'  
    utc_str_time = datetime.utcfromtimestamp(timestamp / 1000.0).strftime('%Y-%m-%d %H:%M:%S.%f')
    return utc_str_time

def request(url):
    response = requests.get(url)
    return response.json()

lasttime = ''

while True:
    r = request(url)
    for i in range(0,len(r['features'])):
        timestamp_to_utc_strtime(r['features'][i]['properties']['time'])[:19]
        if timestamp_to_utc_strtime(r['features'][0]['properties']['time'])[:19] >lasttime:
            print('A %d magnitude earthquake occurred on %s, at %s, Latest earthquakes %s'%(r['features'][0]['properties']['mag'],r['features'][0]['properties']['place'],timestamp_to_utc_strtime(r['features'][0]['properties']['time'])[:19],'https://earthquake.usgs.gov/earthquakes/eventpage/ci38252496#executive'))         
            
            api.PostUpdate('A %d magnitude earthquake occurred on %s, at %s, Latest earthquakes %s'%(r['features'][0]['properties']['mag'],r['features'][0]['properties']['place'],timestamp_to_utc_strtime(r['features'][0]['properties']['time'])[:19],'https://earthquake.usgs.gov/earthquakes/eventpage/ci38252496#executive'), 
                           media=None,
                           media_additional_owners=None, 
                           media_category=None, 
                           in_reply_to_status_id=None, 
                           auto_populate_reply_metadata=False, 
                           exclude_reply_user_ids=None, 
                           latitude=None, 
                           longitude=None, 
                           place_id=None, 
                           display_coordinates=False, 
                           trim_user=False, 
                           verify_status_length=True, 
                           attachment_url=None)         
            
            
            with open('taiwan_earthquake_0828.csv','w',newline='') as f:
                writer = csv.writer(f)
                header = ['Place','Mag','Time']
                writer.writerow(header)
                for i in range(0,len(r['features'])):
                    p = r['features'][i]['properties']
                    writer.writerow([p['place'],p['mag'],timestamp_to_utc_strtime(r['features'][i]['properties']['time'])])
                f.close() 
        
            lasttime = timestamp_to_utc_strtime(r['features'][0]['properties']['time'])[:19] 
        else:
            continue